In [ ]:
pip install langchain sentence-transformers tqdm qdrant-client pandas numpy

In [ ]:
pip install langchain_community

In [ ]:
pip install -U langchain-openai

In [ ]:
!pip install --upgrade openai

In [ ]:
# Importando bibliotecas necessárias
import os
from langchain_openai import OpenAI
from langchain.chains import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import qdrant_client
from qdrant_client.models import PointStruct, VectorParams, Distance
import pandas as pd
import numpy as np
import json
import logging
import concurrent.futures
import re
from collections import Counter

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Upload de arquivos no pc local
from google.colab import files
uploaded = files.upload()

Saving data_cpp.json to data_cpp.json


In [ ]:
import json

# Carrega o arquivo JSON (substitua pelo caminho correto do arquivo JSON)
with open('/content/data_cpp.json') as f:
    posts_data = json.load(f)

# Visualiza o conteúdo do arquivo JSON, incluindo Body, CreationDate, Score e ViewCount
for post in posts_data[:5]:  # Exibe os primeiros 5 posts
    print(f"Post ID: {post['postId']}")
    print(f"Title: {post.get('title', 'No Title')}")
    print(f"Tag: {post.get('tagName', 'No Tag')}")
    print(f"Body: {post.get('body', 'No Body')}")
    print(f"Creation Date: {post.get('creationDate', 'No Creation Date')}")
    print(f"Score: {post.get('score', 'No Score')}")
    print(f"View Count: {post.get('viewCount', 'No View Count')}")
    print('-' * 50)

Post ID: 25
Title: How to use the C socket API in C++ on z/OS
Tag: c++
Body: I'm having issues getting the C sockets API to work properly in C++ on z/OS. Although I am including sys/socket.h, I still get compile time errors telling me that AF_INET is not defined. Am I missing something obvious, or is this related to the fact that being on z/OS makes my problems much more complicated? I discovered that there is a #ifdef that I'm hitting. Apparently, z/OS isn't happy unless I define which 'type' of sockets I'm using with: #define _OE_SOCKETS  Now, I personally have no idea what this _OE_SOCKETS is actually for, so if any z/OS sockets programmers are out there (all 3 of you), perhaps you could give me a rundown of how this all works? Test App: #include &lt;sys/socket.h&gt;  int main() {     return AF_INET; }  Compile/Link Output: cxx -Wc,xplink -Wl,xplink -o inet_test inet.C  './inet.C', line 5.16: CCN5274 (S) The name lookup for 'AF_INET' did not find a declaration. CCN0797(I) Compilatio

In [ ]:
# Cria um DataFrame a partir dos dados JSON
df = pd.DataFrame(posts_data)

# Substituir valores NaN por strings vazias em todas as colunas
df = df.fillna('')

# Garantir que todos os valores no DataFrame sejam strings
df = df.astype(str)

# Criar os textos com postId corretamente
texts = []
for index, row in df.iterrows():
    if row['body']:  # Verifica se o corpo não está vazio
        texts.append({
            'postId': row['postId'],
            'text': f"Post ID: {row['postId']} Title: {row['title']} Body: {row['body']}"
        })
    else:
        print(f"Post ID {row['postId']} não possui corpo.")

# Visualiza os primeiros 5 textos gerados com postId
for text in texts[:5]:
    print(text)

{'postId': '25', 'text': "Post ID: 25 Title: How to use the C socket API in C++ on z/OS Body: I'm having issues getting the C sockets API to work properly in C++ on z/OS. Although I am including sys/socket.h, I still get compile time errors telling me that AF_INET is not defined. Am I missing something obvious, or is this related to the fact that being on z/OS makes my problems much more complicated? I discovered that there is a #ifdef that I'm hitting. Apparently, z/OS isn't happy unless I define which 'type' of sockets I'm using with: #define _OE_SOCKETS  Now, I personally have no idea what this _OE_SOCKETS is actually for, so if any z/OS sockets programmers are out there (all 3 of you), perhaps you could give me a rundown of how this all works? Test App: #include &lt;sys/socket.h&gt;  int main() {     return AF_INET; }  Compile/Link Output: cxx -Wc,xplink -Wl,xplink -o inet_test inet.C  './inet.C', line 5.16: CCN5274 (S) The name lookup for 'AF_INET' did not find a declaration. CCN0

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

# Inicializa o LLM com a chave da API OpenAI
llm = OpenAI(temperature=0.1, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
qa_prompt = PromptTemplate.from_template(
    "You are an experienced C++ developer and a frequent contributor to Stack Overflow. "
    "Your task is to provide a comprehensive, well-structured, and insightful answer to the question below based on the provided context.\n\n"

    "### Context Information:\n"
    "{context}\n\n"

    "### Question:\n"
    "What are the key benefits, challenges, and best practices associated with migrating from C++{previous_version} to C++{current_version}, as well as performing source code rejuvenation?\n\n"

    "### Instructions:\n"
    "1. Use only the context provided to support your answer, ensuring relevance and clarity.\n"
    "2. Be specific and provide actionable, real-world examples when possible.\n"
    "3. Answer each aspect of the question: benefits, challenges, and best practices.\n"
    "4. Structure your answer clearly, using bullet points or numbered lists.\n"
    "5. If the context does not provide enough information, state 'I don't know'.\n\n"

    "### Response Format:\n"
    "--------------------------------------------------------------\n"
    "**Benefits:**\n"
    "1. [Provide a benefit of migration here.]\n"
    "2. [Provide another benefit of migration here.]\n"
    "3. [Provide a third benefit of migration here.]\n\n"

    "**Challenges:**\n"
    "1. [Provide a challenge of migration here.]\n"
    "2. [Provide another challenge of migration here.]\n"
    "3. [Provide a third challenge of migration here.]\n\n"

    "**Best Practices for Source Code Rejuvenation:**\n"
    "1. [Provide a practice for source code rejuvenation here.]\n"
    "2. [Provide another practice for source code rejuvenation here.]\n"
    "3. [Provide a third practice for source code rejuvenation here.]\n"

    "--------------------------------------------------------------\n"
    "Ensure that the response is concise, actionable, and well-formatted.\n\n"
)


In [ ]:
# Criação do llm_chain
llm_chain = LLMChain(llm=llm, prompt=qa_prompt)

<ipython-input-14-2f42845a59c5>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=qa_prompt)


In [ ]:
# Carregar o modelo de embedding
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Gerar embeddings em lotes usando processamento paralelo //tentativa de diminuir o tempo de embeddings
def batch_encode_parallel(texts, model, batch_size=256):
    embeddings = []
    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Gerar embeddings em paralelo
            futures = {executor.submit(model.encode, texts[i:i + batch_size]): i for i in range(0, len(texts), batch_size)}
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
                batch_embeddings = future.result()
                embeddings.append(batch_embeddings)
        print("Embeddings gerados com sucesso.")
        return np.vstack(embeddings)
    except Exception as e:
        print(f"Erro ao gerar embeddings: {str(e)}")
        return None

embeddings = batch_encode_parallel(texts, model)

100%|██████████| 151/151 [00:11<00:00, 13.50it/s]

Embeddings gerados com sucesso.


In [ ]:
# Inicializar o cliente do Qdrant
client = qdrant_client.QdrantClient(":memory:")
collection_name = "relevant_posts"

In [ ]:
# Verificar se a coleção já existe e recriar se necessário
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE),
)

<ipython-input-83-acc8e7a40ee1>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [ ]:
# Adicionar documentos ao Qdrant
points = [
    PointStruct(id=i, vector=embedding.tolist(), payload={"text": texts[i]['text'], "postId": texts[i]['postId']})
    for i, embedding in enumerate(embeddings)
]
client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# Função para realizar a busca e recuperar os documentos mais relevantes
def retrieve(query, client, collection_name, embedding_model, k=10):
    try:
        # Gera o vetor de consulta usando o modelo de embeddings
        query_embedding = embedding_model.encode(query).tolist()

        # Realiza a busca na coleção
        search_result = client.search(
            collection_name=collection_name,
            query_vector=query_embedding,
            limit=k,
            search_params={"hnsw_ef": 100}  # Parâmetro HNSW
        )

        documents = []

        # Adicionar documentos
        for hit in search_result:
            # Verifique se o postId existe no payload
            post_id = hit.payload.get("postId")  # Usando .get para evitar KeyError
            if post_id is None:
                print("Aviso: 'postId' não encontrado no payload!")
            documents.append(Document(page_content=hit.payload["text"], metadata={"postId": post_id}))

        return documents
    except Exception as e:
        print(f"Erro ao buscar documentos: {str(e)}")
        return []

In [ ]:
# Função para filtrar o contexto com base na relevância
def filter_context(context, query, threshold=0.5):
    # Contar a frequência das palavras na consulta
    query_words = query.lower().split()
    context_sentences = context.split('. ')

    # Calcular a relevância de cada sentença
    relevance_scores = []
    for sentence in context_sentences:
        score = sum(Counter(query_words)[word] for word in sentence.lower().split())
        relevance_scores.append((sentence, score))

    # Filtrar sentenças com base na pontuação de relevância
    filtered_context = [sentence for sentence, score in relevance_scores if score >= threshold]

    return ' '.join(filtered_context)

In [ ]:
def split_context(context_list, max_len=500):
    try:
        chunks = []
        for entry in context_list:
            text = entry.page_content  # Acessando o texto
            post_id = entry.metadata.get("postId")

            if not post_id:
                print("Aviso: 'postId' não encontrado no metadata!")

            sentences = re.split(r'(?<=[.!?]) +', text)
            chunk = ''

            for sentence in sentences:
                if len(chunk) + len(sentence) + 1 <= max_len:
                    chunk += sentence + ' '
                else:
                    chunks.append({'postId': post_id, 'text': chunk.strip()})
                    chunk = sentence + ' '

            if chunk:
                chunks.append({'postId': post_id, 'text': chunk.strip()})

        return chunks
    except Exception as e:
        print(f"Erro ao dividir o contexto: {str(e)}")
        return []

In [ ]:
# Função para remover redundâncias nas respostas
def remove_redundancies(responses):
    unique_responses = list(set(responses))
    return unique_responses

In [ ]:
def process_and_format_response(chunks, llm_chain, previous_version, current_version):
    """
    Processa os chunks e gera uma resposta organizada com a associação dos Post IDs.
    """
    try:
        # Inicialização de variáveis para armazenar os conteúdos de benefícios, desafios e práticas
        benefits = []
        challenges = []
        best_practices = []
        used_chunks = []

        # Processa os chunks e gera a resposta
        for chunk in chunks:
            post_id = chunk["postId"]
            context = chunk["text"]

            # Usando o modelo para gerar as partes da resposta
            response = llm_chain.run(
                context=context,
                previous_version=previous_version,
                current_version=current_version
            ).strip()

            # Verifica se a resposta contém informações sobre benefícios, desafios ou melhores práticas
            if "benefits" in response.lower():
                benefits.append(f"Post ID {post_id}: {response}")
            elif "challenges" in response.lower():
                challenges.append(f"Post ID {post_id}: {response}")
            elif "best practices" in response.lower():
                best_practices.append(f"Post ID {post_id}: {response}")

            # Adiciona o Post ID à lista de chunks usados
            used_chunks.append(post_id)

        # Formatação da resposta final
        formatted_response = format_final_response(benefits, challenges, best_practices, used_chunks)
        return formatted_response

    except Exception as e:
        return f"Erro ao processar os chunks: {str(e)}"

In [ ]:
def format_final_response(benefits, challenges, best_practices, used_chunks):
    """
    Formata a resposta final de forma legível e com todos os Post IDs utilizados.
    """
    output = []

    output.append("=" * 40)
    output.append("Resposta Gerada:")

    # Adiciona os benefícios
    if benefits:
        output.append("Benefícios:")
        for benefit in benefits:
            output.append(f"- {benefit}")

    # Adiciona os desafios
    if challenges:
        output.append("Desafios:")
        for challenge in challenges:
            output.append(f"- {challenge}")

    # Adiciona as melhores práticas
    if best_practices:
        output.append("Melhores Práticas:")
        for practice in best_practices:
            output.append(f"- {practice}")

    # Exibe os chunks usados
    output.append("\nPost IDs dos Chunks Utilizados:")
    for post_id in used_chunks:
        output.append(f"- Post ID: {post_id}")

    output.append("=" * 40)
    return "\n".join(output)

In [ ]:
def generate_answer_with_metadata(query, client, collection_name, llm_chain, embedding_model, previous_version, current_version):
    try:
        print("Iniciando recuperação de contexto...")
        # Recuperar contexto relevante
        context = retrieve(query, client, collection_name, embedding_model)

        if not context:
            print("Nenhum contexto relevante encontrado para a consulta.")
            return "Nenhum contexto relevante encontrado para a consulta."

        print(f"{len(context)} documentos recuperados.")

        # Dividir o contexto em chunks
        chunks = split_context(context)
        print(f"Total de {len(chunks)} chunks criados a partir do contexto.")

        # Processar chunks paralelamente
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = list(executor.map(
                lambda chunk: process_chunk_with_metadata(chunk, llm_chain, previous_version, current_version),
                chunks
            ))

        print("Processamento dos chunks concluído.")

        # Organizar os chunks e respostas
        chunks_with_metadata = []
        respostas = []

        for chunk, result in zip(chunks, results):
            # Criar o formato visual para o chunk
            chunk_visual = f"### **Post ID {chunk['postId']}**:\n\n{chunk['text']}\n"
            chunk_visual += f"{'-' * 50}\n"  # Linha de separação para clareza

            # Adicionar a resposta se não for nula
            if result['response']:
                response_visual = f"**Resposta para o Post ID {chunk['postId']}**:\n\n"

                # Benefícios
                response_visual += "**Benefícios:**\n"
                for i, benefit in enumerate(result['response']['benefits'], 1):
                    response_visual += f"{i}. {benefit}\n"

                # Desafios
                response_visual += "\n**Desafios:**\n"
                for i, challenge in enumerate(result['response']['challenges'], 1):
                    response_visual += f"{i}. {challenge}\n"

                # Melhores Práticas
                response_visual += "\n**Melhores Práticas:**\n"
                for i, practice in enumerate(result['response']['best_practices'], 1):
                    response_visual += f"{i}. {practice}\n"

                response_visual += f"{'-' * 50}\n"  # Linha de separação para clareza

                # Adiciona a resposta formatada
                respostas.append(response_visual)

            # Adiciona o chunk formatado
            chunks_with_metadata.append(chunk_visual)

        # Garantir que os dados foram processados
        if not chunks_with_metadata:
            print("Nenhum chunk foi processado.")
        if not respostas:
            print("Nenhuma resposta foi gerada.")

        # Montar resposta final (com metadados e respostas)
        final_response = "\n".join(chunks_with_metadata) + "\n" + "\n".join(respostas)
        return final_response
    except Exception as e:
        print(f"Erro ao gerar resposta com metadados: {str(e)}")
        return "Ocorreu um erro ao gerar a resposta."

In [ ]:
# Função para realizar a busca e recuperar os documentos mais relevantes
def search_context(query, client, collection_name, embedding_model, k=5):
    try:
        # Gera o vetor de consulta usando o modelo de embeddings
        query_embedding = embedding_model.encode(query).tolist()

        # Realiza a busca na coleção
        search_result = client.search(
            collection_name=collection_name,
            query_vector=query_embedding,
            limit=k,
            search_params={"hnsw_ef": 100}
        )

        documents = []
        for hit in search_result:
            documents.append({
                'postId': hit.id,
                'text': hit.payload['text']
            })

        return documents
    except Exception as e:
        print(f"Erro ao buscar contexto: {str(e)}")
        return []

In [ ]:
previous_version = "C++11"
current_version = "C++14"

In [ ]:
# Exemplo de uso da função para gerar uma resposta
query = (
    "What are the main benefits and challenges of migrating from C++11 to C++14?\n\n"
    "What are the prevalence trends in discussions about this migration?\n\n"
    "What practices do developers often use while carrying out source code rejuvenation efforts?"
)
response = generate_answer_with_metadata(query, client, collection_name, llm_chain, model, previous_version, current_version)

Iniciando recuperação de contexto...
10 documentos recuperados.
Total de 25 chunks criados a partir do contexto.
Processando chunk com Post ID: 1931218
Processando chunk com Post ID: 1931126
Processando chunk com Post ID: 1931126
Processando chunk com Post ID: 492014Processando chunk com Post ID: 492014

Processando chunk com Post ID: 492014
Resposta gerada para Post ID 1931126: benefits:
1. Improved performance: C++14 introduces new features such as constexpr functions and variable templates, which can significantly improve the performance of code compared to C++11.
2. Enhanced readability: C++14 introduces new features such as generic lambdas and auto return type deduction, which can make code more concise and easier to read.
3. Better compatibility: C++14 is backward compatible with C++11, meaning that code written in C++11 can still be compiled and run in C++14 without any major changes.

challenges:
1. Learning curve: Migrating from C++11 to C++14 may require developers to learn n